In [40]:
import copy

class WumpusPuzzle:
    def __init__(self, s, a, q, qa, p):
        self.size = s
        self.arrows = a
        self.query = q
        self.queryAnswer = qa
        self.path = p

In [41]:
class Clause:
    def __init__(self, l):
        #list of literals
        self.literals = l

    def __str__(self):
        finalStr = ""
        for i in range(len(self.literals)):
            finalStr += str(i) +". " + str(self.literals[i].negation) + " function " + str(self.literals[i].predicateFunc) + ", variables: "
            for j in range(len(self.literals[i].terms)):
                finalStr += str(self.literals[i].terms[j]) + " "
            finalStr += "\n"
        return finalStr


In [42]:
class Literal:
    def __init__(self, n, f, t):
        #boolean
        self.negation = n
        #predicate function
        self.predicateFunc = f
        #list of terms
        self.terms = t


    def __eq__(self, otherLiteral):
        if(self.negation == otherLiteral.negation and self.predicateFunc == otherLiteral.predicateFunc and termsEqual(self, otherLiteral)):
            return True
        return False

    def replaceVariables(self, dict):
        for replacementVariable in dict.keys():
            for predVariable in range(len(self.terms)):
                if(self.terms[predVariable].value == replacementVariable):
                    print(self.terms[predVariable].value + " being replaced by " + dict[replacementVariable])
                    self.terms[predVariable].value = dict[replacementVariable]


def literalsCancel(literal1, literal2):
    if(literal1.negation != literal2.negation and literal1.predicateFunc == literal2.predicateFunc):
        return True
    else:
        return False

def termsEqual(literal1, literal2):
    for i in range(len(literal1.terms)):
            if literal1.terms[i] != literal2.terms[i]:
                return False
    return True

In [43]:
class Term:
    def __init__(self, v,c,s, value):
        self.isVariable = v
        self.isConstant = c
        self.isSkolem = s
        self.value = value

    def __str__(self):
        return self.value

    def __eq__(self, other):
        if(self.value == other.value):
            return True
        return False

    def __hash__(self):
        return hash(self.value)

In [44]:
class SkolemFunction:
    def __init__(self, term, function):
        self.term = term
        self.function = function

In [45]:
# class Predicate:
#     def __init__(self, v, func):
#         self.variables = v
#         self.function = func
#         self.truthValue = func(v)
#
#     def equalsOther(self, otherPredicate):
#         if(self.function == otherPredicate.function):
#             return True
#         return False
#
#     def replaceVariables(self, dict):
#         for replacementVariable in dict.keys():
#             for predVariable in range(len(self.variables)):
#                 print()
#                 if(self.variables[predVariable] == replacementVariable):
#                     print(self.variables[predVariable])
#                     self.variables[predVariable] = dict[replacementVariable]
#                     print(self.variables[predVariable])


In [46]:
def resolution(KB, query):
    clauses = copy.deepcopy(KB)
    clauses.add(query)
    new = set()
    while(True):
        clausePairs = set()
        #setting up all pairs of clauses
        for clause in clauses:
            for clause2 in clauses:
                if(clause != clause2):
                    clausePairs.add(tuple([clause, clause2]))
        for pair in clausePairs:
            print("clauses before: ", pair[0], pair[1])
            resolvent = resolve(pair[0], pair[1])
            print("clause after: ", resolvent)
            if(len(resolvent.literals) == 0):
                return True
            else:
                new.add(resolvent)
        if(new.issubset(clauses)):
            return False
        clauses = clauses.union(new)



def resolve(c1, c2):
    clause1 = copy.deepcopy(c1)
    clause2 = copy.deepcopy(c2)
    newLiterals = []
    newLiterals += clause1.literals + clause2.literals
    for literal1 in clause1.literals:
        for literal2 in clause2.literals:
            if literalsCancel(literal1, literal2):
                theta = {}
                theta = unify(literal1, literal2, theta)
                print("theta:", theta)
                if(theta != "fail"):
                    #replacing with substitutions
                    for literalSub1 in clause1.literals:
                        for literalSub2 in clause2.literals:
                            literalSub1.replaceVariables(theta)
                            literalSub2.replaceVariables(theta)
                    newLiterals.remove(literal1)
                    newLiterals.remove(literal2)


    newClause = Clause(newLiterals)
    return newClause







In [47]:
# Perform substitution to make x and y the same
def unify(x, y, theta):
    if theta == "fail":
        return "fail"

    if (x == y):
        return theta

    elif (isinstance(x,Term) and x.isVariable):
        return unifyVar(x, y, theta)
    elif (isinstance(y,Term) and y.isVariable):
        return unifyVar(y, x, theta)


    elif isinstance(x, Literal) and isinstance(y, Literal):
        if(x.predicateFunc == y.predicateFunc):
            return unify(x.terms, y.terms, theta)
        else:
            return "fail"


    elif isinstance(x, list) and isinstance(y, list):
        if len(x) != len(y):
            return "fail"
        for i, j in zip(x, y):
            theta = unify(i, j, theta)
            if theta == "fail":
                return "fail"
        return theta

    else:
        return "fail"


def unifyVar(var, x, theta):
    if var in theta:
        return unify(theta[var],x,theta)
    elif x in theta:
        return unify(var,theta[x],theta)
    #Maybe need to have the occur-check call here and write occur-check function? Says not always done in the textbook
    else:
        new = theta.copy()
        new[var.value] = x.value
        return new


In [48]:
PuzzleFile = 'Caves/_wumpus_paths_/easy/path_e1.txt'

def processFile(fileName):
    with open(fileName, 'r', encoding='utf-8-sig') as file:
        content = file.read().strip()
        result = content.split("\n")
        result2 = result[0].split("GRID: ")
        xAxis = int(result2[1].split("x")[0])
        yAxis = int(result2[1].split("x")[1])
        size = [xAxis, yAxis]
        result3 = result[1].split("ARROWS: ")
        arrows = int(result3[1])
        index = 3
        path = result[index]
        pathArr = []
        while(path != ''):
            pathStr = path.strip().split("(")[1].split(")")
            xVal = int(pathStr[0].split(",")[0])
            yVal = int(pathStr[0].split(",")[1])
            breezeVal = pathStr[1].split("B:")[1][0]
            stenchVal = pathStr[1].split("S:")[1][0]
            pathVal = [[xVal, yVal], breezeVal, stenchVal]
            pathArr.append(pathVal)
            index+=1
            path = result[index]
        queryResult = result[index+1].split("QUERY: ")[1].split(',')
        query = [int(queryResult[0][1:]), int(queryResult[1][:-1])]
        resolution = result[index+2].split("RESOLUTION:")[1].strip()
        puzzle = WumpusPuzzle(size, arrows, query, resolution, pathArr)
        print(size)
        print(arrows)
        print(pathArr)
        print(query)
        print(resolution)
        return puzzle

puzzle = processFile(PuzzleFile)

[5, 5]
2
[[[0, 0], 'F', 'F'], [[1, 0], 'F', 'T'], [[1, 1], 'F', 'F'], [[0, 1], 'F', 'T']]
[0, 2]
UNSAFE


In [49]:


KB = set()

Tuna = Term(False, True, False, "Tuna")
Jack = Term(False, True, False, "Jack")
Curiosity = Term(False, True, False, "Curiosity")
x = Term(True, False, False, "x")
f = SkolemFunction(x, "F")
fx = Term(False,False,True, f)
g = SkolemFunction(x, "G")
gx = Term(False,False,True, g)
y = Term(True, False, False, "y")
z = Term(True, False, False, "z")




l = Literal(True, "Cat", [Tuna])
rule6 = Clause([l])

l = Literal(False, "Cat", [x])
l2 = Literal(True, "Animal", [x])
#test unification on different functions
rule7 = Clause([l,l2])


l = Literal(True,"Animal", [fx])
l2 = Literal(True, "Loves", [gx, x])
rule1 = Clause([l,l2])



l = Literal(False, "Loves", [x,fx])
l2 = Literal(True, "Loves", [gx, x])
#test unification on skolems
rule2 = Clause([l,l2])


l = Literal(False, "Loves",[y,x])
l2 = Literal(False, "Animal", [z])
l3 = Literal(False, "Kills", [x,z])
rule3 = Clause([l,l2,l3])


l = Literal(False, "Animal", [x])
l2 = Literal(True, "Loves", [Jack, x])
rule4 = Clause([l,l2])

l = Literal(True, "Kills", [Jack, Tuna])
l2 = Literal(True, "Kills", [Curiosity, Tuna])
#test unification on constants
rule5 = Clause([l,l2])

KB.add(rule1)
KB.add(rule2)
KB.add(rule3)
KB.add(rule4)
KB.add(rule5)
KB.add(rule6)
KB.add(rule7)


l = Literal(False, "Kills", [Curiosity, Tuna])
query = Clause([l])

#Testing literal cancel checking function
# p = Predicate([Jack,Tuna], Kills)
# l = Literal(p, True)
# p = Predicate([Jack,Tuna], Kills)
# l2 = Literal(p, False)
# print(literalsCancel(l,l2))

#Testing unification
# Variables
theta = {}
x2 = copy.deepcopy(x)
theta = unify(x, x2, theta)
print(theta)

# Literals
l = Literal(True, "Kills", [Curiosity, Tuna])
l2 = Literal(True, "Kills", [x, y])
theta = {}
theta = unify(l, l2, theta)
print(theta)
# --- Test ---
# theta = {}
# print("Unify Literals:", unify(l1, l2, theta))
# print("Unify Predicates:", unify(p1, p2, theta))
# print("Unify Clauses:")
# for l in rule7.literals:
#     for l2 in rule6.literals:
#         print(unify(l, l2, theta))
#
#
# #
newClauses = resolve(rule6, rule7)
# # print(rule6)
# # print(rule7)
print(newClauses)
print(rule3)
newerClauses = resolve(newClauses, rule3)
print(newerClauses)
newerClauses2 = resolve(rule5, query)
print(newerClauses2)
#tester = resolution(KB, query)
final = resolve(newerClauses, newerClauses2)
print(final)

{}
{'x': 'Curiosity', 'y': 'Tuna'}
theta: {'x': 'Tuna'}
x being replaced by Tuna
0. True function Animal, variables: Tuna 

0. False function Loves, variables: y x 
1. False function Animal, variables: z 
2. False function Kills, variables: x z 

theta: {'Tuna': 'z'}
Tuna being replaced by z
0. False function Loves, variables: y x 
1. False function Kills, variables: x z 

theta: fail
theta: {}
0. True function Kills, variables: Jack Tuna 

theta: {'x': 'Jack', 'z': 'Tuna'}
x being replaced by Jack
z being replaced by Tuna
0. False function Loves, variables: y Jack 



In [ ]:
#Rules being setup for the wumpus world
#1. All locations visited don't have a wumpus or pit in them
#2. All locations visited with no b/s have no wumpuses or pits in the cardinal directions
#3. ALl locations visited with a b/s have a wumpus or pit in the cardinal direction
KB = set()
for i in range(len(puzzle.path)):
    cell = puzzle.path[i][0]
    breeze = puzzle.path[i][1]
    stench = puzzle.path[i][2]
    #Rule 1
    t = Term(False, True, False, cell)
    literal = Literal(False, "Wumpus", [t])
    clause = Clause([literal])
    KB.add(clause)
    literal = Literal(False, "Pit", [t])
    clause = Clause([literal])
    KB.add(clause)
    #Rule 2/3
    if(breeze == "F"):
        literalList = []
        for x in [cell[0]-1, cell[0]+1]:
            if(x>=0):
                newCell = [x,cell[1]]
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Pit", [t])
                literalList.append(literal)
        for y in [cell[1]-1, cell[1]+1]:
            if(y>=0):
                newCell = [cell[0],y]
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Pit", [t])
                literalList.append(literal)
        clause = Clause(literalList)
        KB.add(clause)

    else:
        literalList = []
        for x in [cell[0]-1, cell[0]+1]:
            if(x>=0):
                newCell = [x,cell[1]]
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Pit", [t])
                literalList.append(literal)
        for y in [cell[1]-1, cell[1]+1]:
            if(y>=0):
                newCell = [cell[0],y]
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Pit", [t])
                literalList.append(literal)
        clause = Clause(literalList)
        KB.add(clause)

    if(stench == "F"):
        literalList = []
        for x in [cell[0]-1, cell[0]+1]:
            if(x>=0):
                newCell = [x,cell[1]]
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Wumpus", [t])
                literalList.append(literal)
        for y in [cell[1]-1, cell[1]+1]:
            if(y>=0):
                newCell = [cell[0],y]
                t = Term(False, True, False, newCell)
                literal = Literal(False, "Wumpus", [t])
                literalList.append(literal)
        clause = Clause(literalList)
        KB.add(clause)

    else:
        literalList = []
        for x in [cell[0]-1, cell[0]+1]:
            if(x>=0):
                newCell = [x,cell[1]]
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Wumpus", [t])
                literalList.append(literal)
        for y in [cell[1]-1, cell[1]+1]:
            if(y>=0):
                newCell = [cell[0],y]
                t = Term(False, True, False, newCell)
                literal = Literal(True, "Wumpus", [t])
                literalList.append(literal)
        clause = Clause(literalList)
        KB.add(clause)

    t = Term(False, True, False, cell)
print(puzzle.path[i])